In [13]:
from flask import Flask, render_template, request
import os
import tensorflow as tf
import numpy as np
import cv2

app = Flask(__name__)

# Load your machine learning model
model_path = "C:/Users/mayan/Desktop/Projects/AI ML/Alzheimer's Detection/final_model_02.h5"
model = tf.keras.models.load_model(model_path)

# Allow files with extension png, jpg, and jpeg
ALLOWED_EXT = {'jpg', 'jpeg', 'png'}

def allowed_file(filename):
    return '.' in filename and filename.rsplit('.', 1)[1].lower() in ALLOWED_EXT

# Function to generate Grad-CAM dynamically
def generate_grad_cam(img_array, class_index, layer_name):
    grad_cam_model = tf.keras.models.Model(inputs=model.input, outputs=model.get_layer(layer_name).output)

    with tf.GradientTape() as tape:
        last_conv_layer = grad_cam_model.get_layer(layer_name)
        iterate_model = tf.keras.models.Model([grad_cam_model.inputs], [grad_cam_model.output, last_conv_layer.output])
        model_out, last_conv_layer_out = iterate_model(img_array)
        class_out = model_out[:, class_index]

    grads = tape.gradient(class_out, last_conv_layer_out)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    last_conv_layer_output = last_conv_layer_out[0]

    heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)

    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    
    return heatmap.numpy()

# Function to overlay Grad-CAM on the original image
def overlay_grad_cam(img_path, heatmap):
    img = cv2.imread(img_path)
    heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))
    heatmap = np.uint8(255 * heatmap)
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
    
    superimposed_img = heatmap * 0.4 + img

    grad_path = 'static/images/grad/grad_cam_result.jpg'
    cv2.imwrite(grad_path, superimposed_img)

    return grad_path

# Function to load and prepare the image in the right shape
def read_image(filename):
    img = tf.keras.preprocessing.image.load_img(filename, target_size=(150, 150))
    x = tf.keras.preprocessing.image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = tf.keras.applications.vgg16.preprocess_input(x)
    return x

@app.route('/')
def index_view():
    return render_template('index.html')

@app.route('/predict', methods=['POST'])
def predict():
    if request.method == 'POST':
        file = request.files['file']
        if file and allowed_file(file.filename):  # Checking file format
            filename = file.filename
            file_path = os.path.join('static/images/input', filename)
            file.save(file_path)
            img = read_image(file_path)  # Preprocessing method
            class_prediction = model.predict(img)
            confidence = np.max(class_prediction) * 100
            classes_x = np.argmax(class_prediction, axis=1)
            stages = ["Mild Demented", "Moderate Demented", "Non Demented", "Very Mild Demented"]
            stage = stages[classes_x[0]]
            
            # Generate Grad-CAM dynamically
            layer_name = 'conv2d_2'
            heatmap = generate_grad_cam(img, classes_x[0], layer_name)
            
            # Overlay Grad-CAM on the original image
            grad_path = overlay_grad_cam(file_path, heatmap)
            
            return render_template('predict.html', stage=stage, prob=confidence, user_image=file_path, grad_image=grad_path)
        else:
            return "Unable to read the file. Please check file extension"

if __name__ == '__main__':
    app.run(debug=True, use_reloader=False, port=8000)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:8000
Press CTRL+C to quit
127.0.0.1 - - [20/Feb/2024 09:21:18] "GET / HTTP/1.1" 200 -


1/1 [==============================] - 0s 134ms/step


127.0.0.1 - - [20/Feb/2024 09:21:25] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [20/Feb/2024 09:21:25] "GET /static/images/input/test1_MOD.jpg HTTP/1.1" 200 -
127.0.0.1 - - [20/Feb/2024 09:21:25] "GET /static/images/grad/grad_cam_result.jpg HTTP/1.1" 200 -
